### Exercise 3 - Solution

You are given three JSON files, representing a default set of settings, and environment specific settings.
The files are included in the downloads, and are named:
* `common.json`
* `dev.json`
* `prod.json`

Your goal is to write a function that has a single argument, the environment name, and returns the "combined" dictionary that merges the two dictionaries together, with the environment specific settings overriding any common settings already defined.

For simplicity, assume that the argument values are going to be the same as the file names, without the `.json` extension. So for example, `dev` or `prod`.

The wrinkle: We don't want to duplicate data for the "merged" dictionary - use `ChainMap` to implement this instead.

In [5]:
import json

def load_settings(env):
    with open(f'{env}.json') as f:
        settings = json.load(f)
    return settings

In [4]:
from pprint import pprint

In [10]:
pprint(load_settings('common'))

{'data': {'input_root': '/default/path/inputs',
          'numerics': {'precision': 6, 'type': 'Decimal'},
          'output_root': '/default/path/outputs'},
 'database': {'db_name': 'deepdive', 'port': 5432, 'schema': 'public'},
 'logs': {'format': '%(asctime)s: %(levelname)s: %(clientip)s %(user)s '
                    '%(message)s',
          'level': 'info'}}


In [11]:
pprint(load_settings('dev'))

{'data': {'input_root': '/dev/path/inputs',
          'numerics': {'type': 'float'},
          'operators': {'add': '__add__'},
          'output_root': '/dev/path/outputs'},
 'database': {'pwd': 'test', 'user': 'test'},
 'logs': {'format': '%(asctime)s: %(levelname)s: %(clientip)s %(user)s '
                    '%(filename)s %(funcName)s %(message)s',
          'level': 'trace'}}


In [12]:
pprint(load_settings('prod'))

{'data': {'input_root': '$DATA_INPUT_PATH', 'output_root': '$DATA_OUTPUT_PATH'},
 'database': {'pwd': '$PG_PWD', 'user': '$PG_USER'}}


In [13]:
from collections import ChainMap

In [16]:
def settings(env):
    common_settings = load_settings('common')
    env_settings = load_settings(env)
    return ChainMap(env_settings, common_settings)

In [17]:
dev = settings('dev')

In [18]:
for k, v in dev.items():
    print(k, ':', v)


data : {'input_root': '/dev/path/inputs', 'output_root': '/dev/path/outputs', 'numerics': {'type': 'float'}, 'operators': {'add': '__add__'}}
database : {'user': 'test', 'pwd': 'test'}
logs : {'level': 'trace', 'format': '%(asctime)s: %(levelname)s: %(clientip)s %(user)s %(filename)s %(funcName)s %(message)s'}


In [19]:
def chain_recursive(d1, d2):
    chain = ChainMap(d1, d2)
    for k, v in d1.items():
        if isinstance(v, dict) and k in d2 and isinstance(d2[k], dict):
            chain[k] = chain_recursive(d1[k], d2[k])
    return chain

In [20]:
d1 = load_settings('common')
d2 = load_settings('dev')

In [23]:
dev = chain_recursive(d2, d1)

In [24]:
pprint(dev)

ChainMap({'data': ChainMap({'input_root': '/dev/path/inputs',
                            'numerics': ChainMap({'type': 'float'},
                                                 {'precision': 6,
                                                  'type': 'Decimal'}),
                            'operators': {'add': '__add__'},
                            'output_root': '/dev/path/outputs'},
                           {'input_root': '/default/path/inputs',
                            'numerics': {'precision': 6, 'type': 'Decimal'},
                            'output_root': '/default/path/outputs'}),
          'database': ChainMap({'pwd': 'test', 'user': 'test'},
                               {'db_name': 'deepdive',
                                'port': 5432,
                                'schema': 'public'}),
          'logs': ChainMap({'format': '%(asctime)s: %(levelname)s: '
                                      '%(clientip)s %(user)s %(filename)s '
                              

In [25]:
d3 = load_settings('prod')

In [26]:
prod = chain_recursive(d3, d1)

In [27]:
pprint(prod)

ChainMap({'data': ChainMap({'input_root': '$DATA_INPUT_PATH',
                            'output_root': '$DATA_OUTPUT_PATH'},
                           {'input_root': '/default/path/inputs',
                            'numerics': {'precision': 6, 'type': 'Decimal'},
                            'output_root': '/default/path/outputs'}),
          'database': ChainMap({'pwd': '$PG_PWD', 'user': '$PG_USER'},
                               {'db_name': 'deepdive',
                                'port': 5432,
                                'schema': 'public'})},
         {'data': {'input_root': '/default/path/inputs',
                   'numerics': {'precision': 6, 'type': 'Decimal'},
                   'output_root': '/default/path/outputs'},
          'database': {'db_name': 'deepdive', 'port': 5432, 'schema': 'public'},
          'logs': {'format': '%(asctime)s: %(levelname)s: %(clientip)s '
                             '%(user)s %(message)s',
                   'level': 'info'}})


In [28]:
prod['logs']['level']

'info'

In [30]:
dev['logs']['level']

'trace'

In [31]:
def settings(env):
    common_settings = load_settings('common')
    env_settings = load_settings(env)
    return chain_recursive(env_settings, common_settings)

In [32]:
prod = settings('prod')

In [33]:
pprint(prod)

ChainMap({'data': ChainMap({'input_root': '$DATA_INPUT_PATH',
                            'output_root': '$DATA_OUTPUT_PATH'},
                           {'input_root': '/default/path/inputs',
                            'numerics': {'precision': 6, 'type': 'Decimal'},
                            'output_root': '/default/path/outputs'}),
          'database': ChainMap({'pwd': '$PG_PWD', 'user': '$PG_USER'},
                               {'db_name': 'deepdive',
                                'port': 5432,
                                'schema': 'public'})},
         {'data': {'input_root': '/default/path/inputs',
                   'numerics': {'precision': 6, 'type': 'Decimal'},
                   'output_root': '/default/path/outputs'},
          'database': {'db_name': 'deepdive', 'port': 5432, 'schema': 'public'},
          'logs': {'format': '%(asctime)s: %(levelname)s: %(clientip)s '
                             '%(user)s %(message)s',
                   'level': 'info'}})
